In [1]:
import bw2data as bd
from collections import defaultdict
from tqdm import tqdm
from thefuzz import fuzz
import bw_processing as bwp
import numpy as np
import math
import stats_arrays as sa

In [2]:
bd.projects.set_current('GSA for archetypes')

In [3]:
bd.databases

Databases dictionary with 3 object(s):
	biosphere3
	ecoinvent 3.8 cutoff
	swiss consumption 1.0

## Finding liquid fuel combustors

In [4]:
ei = bd.Database("ecoinvent 3.8 cutoff")

There are other combustion processes where fuels are measured in megajoules, these will be addressed later.

In [5]:
flows = ('market for diesel,', 'diesel,', 'petrol,', 'market for petrol,')
liquid_fuels = [x 
                for x in ei 
                if x['unit'] == 'kilogram'
                and((any(x['name'].startswith(flow) for flow in flows) or x['name'] == 'market for diesel'))
               ]
{x['name'] for x in liquid_fuels}

{'diesel, import from RoW',
 'diesel, low-sulfur, import from Europe',
 'diesel, low-sulfur, import from RoW',
 'market for diesel',
 'market for diesel, low-sulfur',
 'market for petrol, 15% ETBE additive by volume, with ethanol from biomass',
 'market for petrol, 4% ETBE additive by volume, with ethanol from biomass',
 'market for petrol, 5% ethanol by volume from biomass',
 'market for petrol, low-sulfur',
 'market for petrol, two-stroke blend',
 'market for petrol, unleaded',
 'petrol, low-sulfur, import from Europe',
 'petrol, unleaded, import from RoW'}

### Look into modelling specifics

In [6]:
# a = liquid_fuels[0]
a = [act for act in ei if 'market for petrol, low-sulfur' in act['name'] and 'Europe without Switzerland'==act['location']][0]
for production in a.production():
    pass

print(production['properties']['carbon content'])

{'amount': 0.865, 'unit': 'dimensionless', 'comment': 'carbon content on a dry matter basis (reserved; not for manual entry)'}


In [7]:
production

Exchange: 1.0 kilogram 'market for petrol, low-sulfur' (kilogram, Europe without Switzerland, None) to 'market for petrol, low-sulfur' (kilogram, Europe without Switzerland, None)>

In [8]:
consumer = list(a.consumers())[0]
consumer

Exchange: 0.0641344042785761 kilogram 'market for petrol, low-sulfur' (kilogram, Europe without Switzerland, None) to 'transport, passenger car, medium size, petrol, EURO 4' (kilometer, RER, None)>

In [9]:
co2 = [x for x in bd.Database('biosphere3') if x['name'] == 'Carbon dioxide, fossil']
co2

['Carbon dioxide, fossil' (kilogram, None, ('air', 'low population density, long-term')),
 'Carbon dioxide, fossil' (kilogram, None, ('air',)),
 'Carbon dioxide, fossil' (kilogram, None, ('air', 'urban air close to ground')),
 'Carbon dioxide, fossil' (kilogram, None, ('air', 'lower stratosphere + upper troposphere')),
 'Carbon dioxide, fossil' (kilogram, None, ('air', 'non-urban air or from high stacks'))]

In [10]:
for exc in consumer.output.biosphere():
    if exc.input in co2:
        print(exc.input)
        break

'Carbon dioxide, fossil' (kilogram, None, ('air', 'urban air close to ground'))


In [11]:
exc

Exchange: 0.207834603648 kilogram 'Carbon dioxide, fossil' (kilogram, None, ('air', 'urban air close to ground')) to 'transport, passenger car, medium size, petrol, EURO 4' (kilometer, RER, None)>

In [12]:
total_co2 = sum(exc['amount'] for exc in consumer.output.biosphere() if exc.input in co2)
total_co2

0.207834603648

In [13]:
production['properties']

{'water in wet mass': {'amount': 0.0, 'unit': 'kg'},
 'carbon content, fossil': {'amount': 0.865,
  'unit': 'dimensionless',
  'comment': 'C-content is 86.5% (ecoinvent v2.2 report 6_IV, Tab 3.13)'},
 'water content': {'amount': 0.0,
  'unit': 'dimensionless',
  'comment': 'water mass/dry mass'},
 'dry mass': {'amount': 1.0, 'unit': 'kg'},
 'price': {'amount': 0.71,
  'unit': 'EUR2005',
  'comment': 'Average calculated from All Grades, Areas and Formulations in USA for 2005. Source: http://www.eia.gov/oil_gas/petroleum/data_publications/wrgp/mogas_history.html'},
 'heating value, net': {'amount': 42.5,
  'unit': 'MJ',
  'comment': 'Based on literature, page 611, Report 17: Jungbluth, N., Chudacoff, M., Dauriat, A., Dinkel, F., Doka, G., Faist Emmenegger, M., Gnansounou, E., Kljun, N., Schleiss, K., Spielmann, M., Stettler, C., Sutter, J. 2007: Life Cycle Inventories of Bioenergy. ecoinvent report No. 17, Swiss Centre for Life Cycle Inventories, Dübendorf, CH.'},
 'wet mass': {'amount':

In [19]:
set([act['name'] for act in ei if "land" in act['name']])

{'anchovy, capture by steel purse seiner and landing whole, fresh',
 'anchovy, capture by wooden purse seiner and landing whole, fresh',
 'cement production, Portland',
 'clear-cutting, grassland to arable land, annual crop',
 'clear-cutting, grassland to arable land, forest, intensive',
 'clear-cutting, grassland to arable land, pasture, man made',
 'clear-cutting, grassland to arable land, perennial crop',
 'clear-cutting, primary forest to arable land, annual crop',
 'clear-cutting, primary forest to arable land, forest, intensive',
 'clear-cutting, primary forest to arable land, pasture, man made',
 'clear-cutting, primary forest to arable land, perennial crop',
 'clear-cutting, secondary forest to arable land, annual crop',
 'clear-cutting, secondary forest to arable land, perennial crop',
 'concrete production, 20MPa, ready-mix, with Portland cement',
 'concrete production, 20MPa, self-construction, with Portland cement',
 'concrete production, 40MPa, ready-mix, with Portland cem

In [24]:
[act for act in ei if "plant" in act['name']]

['heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (megajoule, IE, None),
 'heat and power co-generation, natural gas, combined cycle power plant, 400MW electrical' (megajoule, DE, None),
 'market for anaerobic digestion plant, agricultural' (unit, GLO, None),
 'market for planting, sugarcane' (hectare, GLO, None),
 'electricity production, natural gas, conventional power plant' (kilowatt hour, IT, None),
 'tomato seedling production, in unheated greenhouse, for planting' (unit, RoW, None),
 'electricity production, natural gas, combined cycle power plant' (kilowatt hour, CN-CQ, None),
 'treatment of blast furnace gas, in power plant' (kilowatt hour, CN-SA, None),
 'electricity production, solar tower power plant, 20 MW' (kilowatt hour, US-WECC, None),
 'gas motor production, mini CHP plant' (unit, RoW, None),
 'process-specific burdens, hazardous waste incineration plant' (kilogram, RoW, None),
 'market for gas motor, mini CHP plant' (unit, GLO, N

In [ ]:
[act for act in ei if "plant" in act['name']]

In [ ]:
consumer['amount'] / production['amount'] * production['properties']['carbon content']['amount'] * (12 + 16 * 2) / 12

These numbers don't match because there is a second petrol input. Let's write a validation function.

In [ ]:
consumer.input

In [ ]:
def carbon_fuel_emissions_balanced(activity, fuels, co2):
    """Check that we can rescale carbon in fuel to CO2 emissions by checking their stoichiometry.
    
    Returns a ``bool`."""
    try:
        total_carbon = sum(
            # Carbon content amount is fraction of mass, unitless
            exc['amount'] * exc['properties']['carbon content']['amount'] 
            for exc in activity.technosphere() 
            if exc.input in fuels)
    except KeyError:
        return False
    conversion = 12 / (12 + 16 * 2)
    total_carbon_in_co2 = sum(
        exc['amount'] * conversion
        for exc in activity.biosphere()
        if exc.input in co2
    )
    print(total_carbon, total_carbon_in_co2)
    return math.isclose(total_carbon, total_carbon_in_co2, rel_tol=1e-06, abs_tol=1e-3)

In [ ]:
carbon_fuel_emissions_balanced(consumer.input, liquid_fuels, co2)

In [ ]:
for exc in consumer.input.biosphere():
    if exc.input in co2:
        print(exc.input)

In [ ]:
for exc in consumer.input.technosphere():
    if exc.input in liquid_fuels:
        print(exc.input)

We also need an iterator that will return scaling factors sampled from the uncertainty. Note that we need to add all the carbon together to get the rescaling correct.

In [ ]:
def get_samples_and_scaling_vector(activity, fuels, size=10, seed=None):
    """Draw ``size`` samples from technosphere exchanges for ``activity`` whose inputs are in ``fuels``.
    
    Returns:
        * Numpy indices array with shape ``(len(found_exchanges)),)``
        * Numpy flip array with shape ``(len(found_exchanges,))``
        * Numpy data array with shape ``(size, len(found_exchanges))``
        * Scaling vector with relative total carbon consumption and shape ``(size,)``.
    """
    exchanges = [exc for exc in activity.technosphere() if exc.input in fuels]
    static_total = sum(exc['amount'] * exc['properties']['carbon content']['amount'] for exc in exchanges)
    sample = sa.MCRandomNumberGenerator(
        sa.UncertaintyBase.from_dicts(*[exc.as_dict() for exc in exchanges]), 
        seed=seed
    ).generate(samples=size)
    indices = np.array([(exc.input.id, exc.output.id) for exc in exchanges], dtype=bwp.INDICES_DTYPE)
    carbon_fraction = np.array([exc['properties']['carbon content']['amount'] for exc in exchanges]).reshape((-1, 1))
    carbon_total_per_sample = (sample * carbon_fraction).sum(axis=0).ravel()
    flip = np.ones(indices.shape, dtype=bool)
    assert carbon_total_per_sample.shape == (size,)
    return indices, flip, sample, carbon_total_per_sample / static_total

In [ ]:
get_samples_and_scaling_vector(consumer.output, liquid_fuels)

In [ ]:
def rescale_biosphere_exchanges_by_factors(activity, factors, flows):
    """Rescale biosphere exchanges with flows ``flows`` from ``activity`` by vector ``factors``.
    
    ``flows`` are biosphere flow objects, with e.g. all the CO2 flows, but also other flows such as metals, volatile organics, etc. 
    Only rescales flows in ``flows`` which are present in ``activity`` exchanges.
    
    Assumes the static values are balanced, i.e. won't calculate CO2 emissions from carbon in 
    fuels but just rescales given values.

    Returns: Numpy indices and data arrays with shape (number of exchanges found, len(factors)).
    Returns:
        * Numpy indices array with shape ``(len(found_exchanges)),)``
        * Numpy flip array with shape ``(len(found_exchanges,))``
        * Numpy data array with shape ``(len(found_exchanges), len(factors))``
    """
    indices, data = [], []
    assert isinstance(factors, np.ndarray) and len(factors.shape) == 1
    
    for exc in activity.biosphere():
        if exc.input in flows:
            indices.append((exc.input.id, exc.output.id))
            data.append(factors * exc['amount'])
            
    return np.array(indices, dtype=bwp.INDICES_DTYPE), np.zeros(len(indices), dtype=bool), np.vstack(data)

In [ ]:
i, d = rescale_biosphere_exchanges_by_factors(consumer.output, np.linspace(0.8, 1.2, 10), co2)
i, d

In [ ]:
d.shape